In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'label_anything'))
sys.path.append(str(Path.cwd().parent / 'label_anything' / 'data'))
print(sys.path)

['/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/notebooks', '/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/notebooks', '/home/emanuele/Workspace/pynaoqi', '/usr/local/lib/python38.zip', '/usr/local/lib/python3.8', '/usr/local/lib/python3.8/lib-dynload', '', '/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/venv/lib/python3.8/site-packages', '/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything', '/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/label_anything', '/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/label_anything/data']


In [2]:
import torch
from PIL import Image
from torchvision.transforms import Compose, PILToTensor
from label_anything.data.transforms import CustomNormalize, CustomResize
from label_anything.data.dataset import LabelAnythingDataset
import numpy as np
import matplotlib.pyplot as plt
from label_anything.logger.text_logger import get_logger
from label_anything.logger.utils import (
    extract_boxes_from_tensor,
    image_with_points,
    structure_annotations
)
import comet_ml
from label_anything.logger.image_logger import Logger
from torch.utils.data import DataLoader
import cv2
from torchvision.transforms.functional import resize

In [3]:
comet_ml.init(project_name='label-anything')

In [4]:
experiment = comet_ml.Experiment()
logger = Logger(experiment)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 19:11:38] [setup] RAM Tracking...
[codecarbon INFO @ 19:11:38] [setup] GPU Tracking...
[codecarbon INFO @ 19:11:39] No GPU found.
[codecarbon INFO @ 19:11:39] [setup] CPU Tracking...
[codecarbon WARNING @ 19:11:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:11:39] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz
[codecarbon INFO @ 19:11:39] >>> Tracker's metadata:
[codecarbon INFO @ 19:11:39]   Platform system: Linux-6.5.0-4-amd64-x86_64-with-glibc2.34
[codecarbon INFO @ 19:11:39]   Python version: 3.8.18
[codecarbon INFO @ 19:11:39]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 19:11:39]   Available RAM : 15.323 

In [5]:
preprocess = Compose([
    CustomResize(1024),
    PILToTensor(),
    CustomNormalize(1024)
])

In [6]:
dataset = LabelAnythingDataset(
    instances_path="/home/emanuele/Dottorato/on-going/label_anythings/LabelAnything/label_anything/data/raw/annotations/instances_train2017.json",
    img_dir="/home/emanuele/Downloads/train2017",
    preprocess=preprocess,
)

Loading gt
Using json


[codecarbon INFO @ 19:11:59] Energy consumed for RAM : 0.000027 kWh. RAM Power : 5.746290206909181 W
[codecarbon INFO @ 19:11:59] Energy consumed for all CPUs : 0.000065 kWh. Total CPU Power : 14.0 W
[codecarbon INFO @ 19:11:59] 0.000092 kWh of electricity used since the beginning.


In [7]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=2,
    collate_fn=dataset.collate_fn,
    shuffle=False,
)

In [8]:
batch, gt = next(iter(dataloader))

In [ ]:
for k, v in batch.items():
    if type(v) == list:
        print(f"key: {k}, value: {len(v)}")
    else:
        print(f"key: {k}, value: {v.shape}")

print("gt:", gt.shape)

key: images, value: torch.Size([2, 3, 3, 1024, 1024])
key: prompt_points, value: torch.Size([2, 3, 5, 3, 2])
key: flag_points, value: torch.Size([2, 3, 5, 3])
key: prompt_bboxes, value: torch.Size([2, 3, 5, 5, 4])
key: flag_bboxes, value: torch.Size([2, 3, 5, 5])
key: prompt_masks, value: torch.Size([2, 3, 5, 256, 256])
key: flag_masks, value: torch.Size([2, 3, 5])
key: dims, value: torch.Size([2, 3, 2])
key: classes, value: 2
key: flag_gts, value: torch.Size([2, 5])
gt: torch.Size([2, 3, 640, 640])


[codecarbon INFO @ 19:12:14] Energy consumed for RAM : 0.000050 kWh. RAM Power : 5.746290206909181 W
[codecarbon INFO @ 19:12:14] Energy consumed for all CPUs : 0.000122 kWh. Total CPU Power : 14.0 W
[codecarbon INFO @ 19:12:14] 0.000172 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:12:29] Energy consumed for RAM : 0.000074 kWh. RAM Power : 5.746290206909181 W
[codecarbon INFO @ 19:12:29] Energy consumed for all CPUs : 0.000181 kWh. Total CPU Power : 14.0 W
[codecarbon INFO @ 19:12:29] 0.000255 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:12:44] Energy consumed for RAM : 0.000098 kWh. RAM Power : 5.746290206909181 W
[codecarbon INFO @ 19:12:44] Energy consumed for all CPUs : 0.000239 kWh. Total CPU Power : 14.0 W
[codecarbon INFO @ 19:12:44] 0.000337 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:12:59] Energy consumed for RAM : 0.000122 kWh. RAM Power : 5.746290206909181 W
[codecarbon INFO @ 19:12:59] Energy consumed for a

In [ ]:
batch["images"].shape

In [ ]:
logger.log_batch(0, 0, batch, dataset.categories)

In [ ]:
from label_anything.logger.utils import (
    crop_padding,
    extract_masks_dynamic,
    extract_polygons_from_tensor,
    resize_gt,
)


for k, v in batch.items():
    if type(v) == list:
        print(f"{k}: {len(v)}")
    else:
        print(f"{k}: {v.shape}")

print("gt:", gt.shape)

In [ ]:
# def log_gt(batch_idx, step, input_dict, gt):
batch_idx = 0
step = 0
input_dict = batch

images = input_dict["images"]
dims = input_dict["dims"]
data = []
categories = dataset.categories

In [ ]:
from label_anything.logger.utils import take_image
from label_anything.visualization.visualize import get_image


def get_class_ids(classes):
    res_classes = []
    for c in classes:
        max_len = 0
        max_idx = 0
        for i, x in enumerate(c):
            max_len = max(max_len, len(x))
            if len(x) == max_len:
                max_idx = i
        res_classes.append(list(c[max_idx]))
    return res_classes


classes = get_class_ids(input_dict["classes"])
data = []

for b in range(gt.size(0)):
    print("b", b)
    img = take_image(images[b, 0], dims[b, 0])
    image = get_image(img)
    n_gt = resize_gt(crop_padding(gt[b, 0]).float(), dims[b, 0])
    masks = extract_masks_dynamic(n_gt)
    #print(f"masks: {len(masks)}")
    label = categories[classes[b][0]]["name"]
    print(label)
    for i, mask in enumerate(masks):
        polygons = extract_polygons_from_tensor(mask)
        data.append(
            {
                "points": polygons,
                "label": label,
                "score": None,
            }
        )


annotations = [{"name": "Ground T!", "data": data}]

In [ ]:
len(data)

In [ ]:
logger.log_image(
    image,
    annotations,
)

In [ ]:
logger.log_gt(batch_idx, step, input_dict, gt, categories)
# print(f"image: {image}, annotations: {annotations}")
# logger.log_image(image, annotations)

# for b in range(images.size(0)):
#     image = images[b, 0]
#     img = get_image(image)
#     print(img)
#     logger.log_image(img)
#     for c in range(1, input_dict["prompt_masks"].shape[2]):
#         if c > len(classes[b]):
#             break
#         label = categories[classes[b][c - 1]]["name"]
#         print(label)

In [ ]:
label

In [ ]:

for b in range(images.shape[0]):
    print(b)
    print(dims[b])
    n_gt = resize_gt(crop_padding(gt[b, 0]).float(), dims[b, 0])
    print("n_gt:", n_gt.size())
    masks = extract_masks_dynamic(n_gt)
    print(f"masks: {len(masks)}")
    for i, mask in enumerate(masks):
        polygons = extract_polygons_from_tensor(mask)
        data.append({"points": polygons, "label": label, "score": None})
        print(f"data: {len(data)}")
        print(data[0])
        break
    break
annotations = [{"name": "Ground truth", "data": data}]

In [ ]:
logger.log_image(image, annotations)

In [ ]:
logger.log_gt(batch_idx, step, input_dict, gt)

In [ ]:
type(annotations)
annotations

In [ ]:
def get_image(image_tensor):
    MEAN = np.array([123.675, 116.280, 103.530]) / 255
    STD = np.array([58.395, 57.120, 57.375]) / 255
    unnormalized_image = (image_tensor.numpy() * np.array(STD)[:, None, None]) + np.array(MEAN)[:, None, None]
    unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
    unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
    return Image.fromarray(unnormalized_image)

In [ ]:
mask = batch["prompt_masks"][0, 2, 1]

In [ ]:
mask.unsqueeze(0).shape

In [ ]:
mask = resize(mask.unsqueeze(0), (1024, 1024), interpolation=Image.NEAREST)
mask = np.array(mask).astype(np.uint8)

In [ ]:
mask = mask.squeeze() 

In [ ]:
contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
polygons = []

for obj in contours:
    coords = []
        
    for point in obj:
        coords.append(int(point[0][0]))
        coords.append(int(point[0][1]))

    polygons.append(coords)

In [ ]:
polygons

In [ ]:
image = batch["images"][0, 2]
mask = batch["prompt_masks"][0, 2, 1]
annotations_mask = structure_annotations(
    polygons,
)
logger.log_image(
    get_image(image),
    annotations_mask,
)